# 10. Merge the walk network, bike network, generated crossings and public transport stops

In [ ]:
# Standard library and path imports
import os
import set_path

# Third-party library imports
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import folium

# Local or project-specific imports
import plot_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

## Import networks, crossings and public transport stops

In [ ]:
# Import network and crossings
gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=st.CRS)
gdf_osm_crossings = gpd.read_file(cf.output_osm_crossings).to_crs(crs=st.CRS)
gdf_project_sidewalk_crossings = gpd.read_file(cf.output_project_sidewalk_crossings).to_crs(crs=st.CRS)
gdf_traffic_sign_crossings = gpd.read_file(cf.output_traffic_sign_crossings).to_crs(crs=st.CRS)
gdf_bike_network = gpd.read_file(cf.output_bike_network_cut).to_crs(crs=st.CRS)
gdf_walk_bike_connections = gpd.read_file(cf.output_walk_bike_connections_base + '_max_height_0.04.gpkg').to_crs(crs=st.CRS)
gdf_walk_public_transport_stop_connections = gpd.read_file(cf.output_walk_public_transport_stop_connections).to_crs(crs=st.CRS)
gdf_public_transport_stop_features = gpd.read_file(cf.output_public_transport_features).to_crs(crs=st.CRS)

curb_crossing_filles = [file for file in os.listdir(cf.out_folder) if file.startswith('curb_crossings')]
gdf_curb_crossings = pd.concat([gpd.read_file(cf.out_folder + file).to_crs(crs=st.CRS) for file in curb_crossing_filles])

## Set column values

In [ ]:
# Set crossing and connection columns columns
gdf_osm_crossings['crossing_type'] = 'osm'
gdf_project_sidewalk_crossings['crossing_type'] = 'project_sidewalk'
gdf_traffic_sign_crossings['crossing_type'] = 'traffic_sign'
gdf_curb_crossings['crossing_type'] = 'curb_height'

gdf_crossings = pd.concat([gdf_osm_crossings, gdf_project_sidewalk_crossings, 
                     gdf_traffic_sign_crossings, gdf_curb_crossings])

In [ ]:
# Set edge type columns and length
column = 'crossing'
gdf_network[column] = 'No'
gdf_crossings[column] = 'Yes'
gdf_walk_bike_connections[column] = 'No'
gdf_public_transport_stop_features[column] = 'No'
gdf_walk_public_transport_stop_connections[column] = 'No'

column = 'walk_bike_connection'
gdf_network[column] = 'No'
gdf_bike_network[column] = 'No'
gdf_crossings[column] = 'No'
gdf_walk_bike_connections[column] = 'Yes'
gdf_public_transport_stop_features[column] = 'No'
gdf_walk_public_transport_stop_connections[column] = 'No'

column = 'walk_public_transport_stop_connection'
gdf_network[column] = 'No'
gdf_bike_network[column] = 'No'
gdf_crossings[column] = 'No'
gdf_walk_bike_connections[column] = 'No'
gdf_public_transport_stop_features[column] = 'No'
gdf_walk_public_transport_stop_connections[column] = 'Yes'

column = 'public_transport_stop'
gdf_network[column] = 'No'
gdf_bike_network[column] = 'No'
gdf_crossings[column] = 'No'
gdf_walk_bike_connections[column] = 'No'
gdf_public_transport_stop_features[column] = 'Yes'
gdf_walk_public_transport_stop_connections[column] = 'No'

column = 'path_type'
gdf_network[column] = 'walk'
gdf_bike_network[column] = 'bike'
gdf_crossings[column] = 'walk'
gdf_walk_bike_connections[column] = 'walk_bike_connection'
gdf_walk_public_transport_stop_connections[column] = 'walk'

gdf_crossings['length'] = gdf_crossings.geometry.length
gdf_walk_bike_connections['length'] = gdf_walk_bike_connections.geometry.length
gdf_walk_public_transport_stop_connections['length'] = gdf_walk_public_transport_stop_connections.geometry.length

## Store

In [ ]:
# Merge walking network and crossings
dfs = [gdf_network, gdf_bike_network, gdf_crossings, 
       gdf_walk_bike_connections, gdf_walk_public_transport_stop_connections, 
       gdf_public_transport_stop_features]
non_empty_dfs = [df for df in dfs if not df.empty]
gdf_walk_bike_network_including_crossings = pd.concat([gdf_network, gdf_bike_network, gdf_crossings, 
                                                       gdf_walk_bike_connections, gdf_walk_public_transport_stop_connections, 
                                                       gdf_public_transport_stop_features])

# Export network including crossings
gdf_walk_bike_network_including_crossings.to_file(cf.output_basic_network_including_crossings, driver='GPKG')

## Visualize final network

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.7940192423718443], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

feature_names = gdf_walk_bike_network_including_crossings.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_walk_bike_network_including_crossings, tooltip=tooltip, style_function=lambda x: {"color": "black", "weight": 3}).add_to(map)

map